In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

nvidia-smi 查看cuda情况
dir() 查看包内有什么
help() 查看帮助

In [9]:
from torch.utils.data import Dataset, DataLoader

PyTorch中的torch.utils.data.Dataset类是一个用于表示数据集的抽象类，它定义了访问数据样本及其标签的标准方式。以下是其核心作用和使用方法的详细说明：
Dataset 是一个抽象基类（Abstract Base Class），它定义了 __len__ 和 __getitem__ 这两个必须实现的方法，但具体如何加载数据、从哪里加载数据，完全由用户自定义。

核心作用

数据封装与标准化接口：

Dataset类将数据（如图像、文本等）及其标签封装为一个统一的对象，要求用户实现__len__和__getitem__方法。
__len__：返回数据集的样本总数。
__getitem__：根据索引返回对应的样本和标签（支持张量或其他格式）。

与DataLoader配合：

Dataset通常与DataLoader结合使用。DataLoader负责批量加载数据、多线程加速、数据打乱等操作，而Dataset专注于单个样本的读取和预处理。

灵活性：

内置数据集（如MNIST、CIFAR-10）已实现Dataset接口，可直接使用。
自定义数据集时，继承Dataset并实现关键方法，可处理任意存储格式的数据（如文件夹、CSV文件、数据库）。
实现自定义Dataset的步骤

继承Dataset类：

python
复制代码
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data          # 数据路径或数据本身
        self.labels = labels      # 标签
        self.transform = transform# 数据预处理（如transforms.Compose）

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label


使用DataLoader加载：

python
复制代码
from torch.utils.data import DataLoader
dataset = CustomDataset(data, labels, transform=my_transforms)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

实际应用场景
图像处理：读取图片文件，应用数据增强（随机裁剪、翻转等）。
文本处理：加载文本数据并转换为词向量。
非标准数据：处理自定义存储结构的数据（如数据库查询结果）。
内置工具与优化

TensorDataset：若数据已存储在张量中，可直接使用TensorDataset快速封装。

python
复制代码
import torch
from torch.utils.data import TensorDataset
data_tensor = torch.tensor(...)
labels_tensor = torch.tensor(...)
dataset = TensorDataset(data_tensor, labels_tensor)


数据增强：通过torchvision.transforms模块定义预处理流程，并在__getitem__中应用。

注意事项
线程安全：确保__getitem__的实现是线程安全的（避免全局变量修改）。
高效读取：对于大规模数据（如图像），避免在__init__中一次性加载全部数据，应动态读取（如按需加载图片文件）。

总结来说，Dataset类是PyTorch数据处理流程的基石，通过自定义实现，用户能够灵活适配各种数据源和预处理需求，而DataLoader则在此基础上提供了高效的数据加载和管理功能。

In [14]:
from PIL import Image

In [11]:
class Mydata(Dataset):
    def __init__(self):



Help on class Dataset in module torch.utils.data.dataset:

class Dataset(typing.Generic)
 |  An abstract class representing a :class:`Dataset`.
 |
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`. Subclasses could also
 |  optionally implement :meth:`__getitems__`, for speedup batched samples
 |  loading. This method accepts list of indices of samples of batch and returns
 |  list of samples.
 |
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs an index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral indices

In [12]:
pip install opencv-python

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.8/39.5 MB 7.5 MB/s eta 0:00:06
   ----- ---------------------------------- 5.5/39.5 MB 19.2 MB/s eta 0:00:02
   ----------- ---------------------------- 11.0/39.5 MB 21.9 MB/s eta 0:00:02
   --------------- ------------------------ 15.2/39.5 MB 23.1 MB/s eta 0:00:02
   ----------------------- ---------------- 22.8/39.5 MB 24.2 MB/s eta 0:00:01
   ----------------------------- ---------- 29.4/39.5 MB 25.5 MB/s eta 0:00:01
   ----------------------------------- ---- 34.9/39.5 MB 25.4 MB/s eta 0:00:01
   ---------------------------------------  39.3/39.5 MB 25.3 MB/s eta 0:00:01
   ---------------------------------------- 39.5/39.5 MB 24.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
